# KG Rag

In [3]:
! pip install --qU pandas langchain_neo4j neo4j langchain_core langchain_ollama langchain_groq


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --qU


**Connection to constructed Neo4j graph**

In [4]:
# 1. Importing Required Libraries
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph

# 2. Connect to Neo4j
NEO4J_URI="neo4j+s://07561b92.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="2wQMIQq_1cANjkh837eMSHUsYBH2F-XLialhvPmM3Bw"

URI = NEO4J_URI
AUTH = (NEO4J_USERNAME,NEO4J_PASSWORD) #AUTH = ("<Username>", "<Password>")
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()


graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
print(graph)
print("Connected to Neo4j graph.")

Connected to Neo4j graph.


**Connectivity Check**

In [5]:
graph.query(
"""
MATCH (d:Disease)<-[:HAS_SYMPTOM]-(s:Symptom)
WHERE s.name IN ["itching", "skin_rash", "nodal_skin_eruptions"]
RETURN d.name AS Disease, COUNT(s) AS MatchingSymptoms
ORDER BY MatchingSymptoms DESC
LIMIT 5

""")

[{'Disease': 'Fungal Infection', 'MatchingSymptoms': 3},
 {'Disease': 'Drug Reaction', 'MatchingSymptoms': 2},
 {'Disease': 'Chickenpox', 'MatchingSymptoms': 2},
 {'Disease': 'Jaundice', 'MatchingSymptoms': 1},
 {'Disease': 'Chronic Cholestasis', 'MatchingSymptoms': 1}]

**LLM connection**

In [6]:
import os
from langchain_groq import ChatGroq

# Set your Groq API key
os.environ["GROQ_API_KEY"] = "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"

# Initialize the ChatGroq model
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=1,
    max_tokens=1024,
)


**Connectivity Check**

In [7]:
# Define the conversation
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Who is Abdul kalam in 2 lines?"),
]

# Generate the response
ai_msg = llm.invoke(messages)
print(ai_msg.content)


Abdul Kalam was a renowned Indian scientist and politician who served as the 11th President of India from 2002 to 2007. He is often referred to as the "Missile Man of India" for his significant contributions to the country's space and missile programs.


**Graph RAG: Neo4j Graph + LLM with Prompt for Cypher Query Generation**

In [27]:
from langchain_neo4j import GraphCypherQAChain
from langchain_ollama import ChatOllama
from langchain_core.prompts.prompt import PromptTemplate

# Define the schema and question
schema = """
Nodes:
- Disease: {name: string}
- Symptom: {name: string}
Relationships:
- (Disease)-[:HAS_SYMPTOM]->(Symptom)
"""

question = "What are diseases associated with this symptoms itching, skin_rash, dischromic _patches?"

CYPHER_GENERATION_TEMPLATE = """
Task: Generate a Cypher statement to query a graph database.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

Examples: 
# What diseases are associated with itching, skin_rash, nodal_skin_eruptions?
MATCH (d:Disease)<-[:HAS_SYMPTOM]-(s:Symptom)
WHERE s.name IN ["itching", "skin_rash", "nodal_skin_eruptions"]
RETURN d.name AS Disease, COUNT(s) AS MatchingSymptoms
ORDER BY MatchingSymptoms DESC
LIMIT 5

# What treatments are associated with fever and headache?
MATCH (t:Treatment)<-[:TREATS]-(d:Disease)-[:HAS_SYMPTOM]->(s:Symptom)
WHERE s.name IN ["fever", "headache"]
RETURN t.name AS Treatment, COUNT(s) AS MatchingSymptoms
ORDER BY MatchingSymptoms DESC

# Which doctors are treating diseases related to fatigue and nausea?
MATCH (doc:Doctor)-[:TREATS]->(d:Disease)-[:HAS_SYMPTOM]->(s:Symptom)
WHERE s.name IN ["fatigue", "nausea"]
RETURN doc.name AS Doctor, COUNT(s) AS MatchingSymptoms
ORDER BY MatchingSymptoms DESC
LIMIT 5

The question is:
{question}"""

# # Initialize the LLM
# llm = ChatOllama(model="deepseek-r1:1.5b", temperature=1.5)

# Create the prompt
prompt = PromptTemplate(input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE)

# Set up the GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    cypher_prompt=prompt,
    verbose=True,
    allow_dangerous_requests=True
)

# 11. Invoke the Query Chain and Get the Result
KG_response= chain.invoke({"query": question})

# 12. Print the Result
print(KG_response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease)<-[:HAS_SYMPTOM]-(s:Symptom)
WHERE s.name IN ["itching", "skin_rash", "dischromic_patches"]
RETURN d.name AS Disease, COUNT(s) AS MatchingSymptoms
ORDER BY MatchingSymptoms DESC
LIMIT 5
Full Context:
[{'Disease': 'Chickenpox', 'MatchingSymptoms': 2}, {'Disease': 'Drug Reaction', 'MatchingSymptoms': 2}, {'Disease': 'Fungal Infection', 'MatchingSymptoms': 2}, {'Disease': 'Jaundice', 'MatchingSymptoms': 1}, {'Disease': 'Chronic Cholestasis', 'MatchingSymptoms': 1}]

> Finished chain.
{'query': 'What are diseases associated with this symptoms itching, skin_rash, dischromic _patches?', 'result': 'Based on the symptoms itching, skin_rash, dischromic_patches, the associated diseases are Chickenpox, Drug Reaction, and Fungal Infection, as they have the most matching symptoms. Additionally, Jaundice and Chronic Cholestasis are also possible, although they have fewer matching symptoms.'}


In [ ]:
# 11. Invoke the Query Chain and Get the Result
KG_response= chain.invoke({"query": question})

# 12. Print the Result
print(KG_response)

# Agentic Rag

In [1]:
! pip install -qU duckduckgo-search langchain_tools

**DuckDuckGo Search Tool with LangChain for Disease Recommendation**

In [9]:
# Import necessary modules
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo search tool
ddg_search = DuckDuckGoSearchRun()

# Define the tools the agent can use
tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="Search for diseases based on symptoms using DuckDuckGo"
    )
]

# Initialize the agent with Zero-Shot-React-Description agent type
agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Query the agent to recommend diseases based on the symptoms
Agent_response = agent.invoke("What are top diseases are associated with itching, skin_rash, nodal_skin_eruptions?")

print(Agent_response['output'])

C:\Users\admin\AppData\Local\Temp\ipykernel_9344\2619257943.py:18: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
To recommend diseases based on the given symptoms, I should first search for diseases that are commonly associated with these symptoms. The symptoms provided are itching, skin rash, and nodal skin eruptions. 

Action: DuckDuckGo Search
Action Input: {'query': 'diseases causing itching skin rash nodal skin eruptions', 'verbose': False, 'start_color': 'green', 'color': 'green', 'callbacks': None, 'tags': None, 'metadata': None, 'run_name': None, 'run_id': None, 'config': None, 'tool_call_id': None}
Observation: a rash that appears red, skin-colored, or darker than the natural skin color a rash that spreads from the face down the body 3-5 days after the first symptoms appear tiny white spots with bluish ... Itchy skin is an irritating sensation that makes you want to scratch. It's also called pruritus (proo-RIE-tus). Itchy skin is often caused by dry skin and is common in older adults, as skin tends to become drier with age. Depending on the cause o

# KG Rag response + Agentic Rag Response

In [10]:
import os
import re
from langchain_groq import ChatGroq

# Set your Groq API key
os.environ["GROQ_API_KEY"] = "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"

# Initialize the ChatGroq model
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=1,
    max_tokens=1024,
    stream=True
)

# Define the conversation
messages = [
    ("system", "You are a highly knowledgeable and detail-oriented medical assistant with the ability to critically analyze responses for errors and inconsistencies."),
    ("human", f"I have two sets of responses related to symptoms:\n"
              f"1. Knowledge Graph Response: {KG_response}\n"
              f"2. Web Search Response: {Agent_response}\n"
              f"\n"
              f"Please follow these steps to refine and merge the responses:\n"
              f"1. Review both responses for mistakes, inconsistencies, and redundant diseases. Ensure no disease is listed more than once and prioritize based on severity and commonality.\n"
              f"2. Suggest a healthy diet to manage or alleviate the symptoms. Include specific foods and drinks to consume and avoid.\n"
              f"3. Recommend safe and easy natural techniques or exercises that can complement medical treatment.\n"
              f"4. Specify the type of doctor or specialist to consult and provide a brief reason for each recommendation.\n"
              f"\n"
              f"After reviewing and enhancing the responses, follow these steps:\n"
              f"    1. List all (highly, medium,low) associated diseases, eliminating duplicates and prioritizing based on severity.\n"
              f"    2. Suggest a healthy diet, including specific foods to consume and avoid.\n"
              f"    3. Recommend natural techniques or exercises that are easy and safe.\n"
              f"    4. Specify the type of doctor or specialist to consult with a brief reason.\n"
              f"Ensure the response is clear, concise, and easy to understand."),
]

# # Generate the response
# for chunk in llm.stream(messages):
#     print(chunk.content, end='', flush=True)

# Stream and clean markdown on the fly
for chunk in llm.stream(messages):
    cleaned_chunk = re.sub(r'(\*\*|\*|__|_|\[.*?\]\(.*?\))', '', chunk.content)
    print(cleaned_chunk, end='', flush=True)

c:\Users\admin\OneDrive\Desktop\Symptoms_based_disease_prediction\symp_venv\Lib\site-packages\langchain_groq\chat_models.py:364: UserWarning: WARNING! stream is not default parameter.
                    stream was transferred to model_kwargs.
                    Please confirm that stream is what you intended.
  warnings.warn(


After refining and merging the responses, here are the results:

Associated Diseases:
1. Fungal Infection (e.g., ringworm)
2. Prurigo Nodularis
3. Eczema
4. Lyme disease
5. Staphylococcal scalded skin syndrome (SSS)
6. Poison ivy (contact dermatitis)

These diseases are associated with the symptoms of itching, skin rash, and nodal skin eruptions, prioritized based on severity and commonality.

Healthy Diet:
To manage or alleviate symptoms, consider the following dietary recommendations:
- Consume: omega-3 rich foods (salmon, flaxseeds), probiotics (yogurt, kefir), vitamin E-rich foods (nuts, seeds), and antioxidant-rich foods (berries, leafy greens)
- Avoid: processed foods, sugary drinks, dairy products (for some individuals with eczema or sensitive skin), and foods high in histamine (fermented foods, citrus fruits)

Natural Techniques or Exercises:
Complement medical treatment with these safe and easy natural techniques:
- Moisturize skin regularly
- Apply cool compresses to reduce i